In [42]:
import sys
import math
import pickle
import numpy as np

k = 2
sys.setrecursionlimit(1500)

In [43]:
def inputDigit(name="digitdata/trainingimages"):
    with open(name) as digitTxt:
        image = [list(line)[0:28] for line in digitTxt]
        rows = len(image)
    return image

In [44]:
digitImage = inputDigit()
print(len(digitImage))
print(len(digitImage[139999]))

140000
28


In [45]:
import collections

def readRough(name="digitdata/traininglabels"):
    with open(name) as label:
        labels = []
        for line in label:
            labels.append(line[0])
    return labels


In [46]:
label = readRough()
print (len(label))
#print sorted(collections.Counter(label).items())
class_ = sorted(collections.Counter(label).items())
print (class_)
with open('labelstats.txt', 'wb') as fp:
    pickle.dump(class_, fp)

5000
[('0', 479), ('1', 563), ('2', 488), ('3', 493), ('4', 535), ('5', 434), ('6', 501), ('7', 550), ('8', 462), ('9', 495)]


In [47]:
with open ('labelstats.txt', 'rb') as fp:
    class_ = pickle.load(fp)

digitImage = inputDigit()
training0=np.zeros(shape=(10, 28, 28))
training1=np.zeros(shape=(10, 28, 28))

count = 0
for i in range(5000):
    for row in range(28):
        for col in range(28):
            if digitImage[i*28 + row][col] == ' ':
                training1[int(label[i])][row][col] += 0
                training0[int(label[i])][row][col] += 1
            else:
                training1[int(label[i])][row][col] += 1
                training0[int(label[i])][row][col] += 0

for i in range(10):
    training1[i] = (training1[i] + k) / (class_[i][1] + k * 2)
    training0[i] = (training0[i] + k) / (class_[i][1] + k * 2)
print(training0[0][0])
print(training1[0][0])



[ 0.99585921  0.99585921  0.99585921  0.99585921  0.99585921  0.99585921
  0.99585921  0.99585921  0.99585921  0.99585921  0.99585921  0.99585921
  0.99585921  0.99585921  0.99585921  0.99585921  0.99585921  0.99585921
  0.99585921  0.99585921  0.99585921  0.99585921  0.99585921  0.99585921
  0.99585921  0.99585921  0.99585921  0.99585921]
[ 0.00414079  0.00414079  0.00414079  0.00414079  0.00414079  0.00414079
  0.00414079  0.00414079  0.00414079  0.00414079  0.00414079  0.00414079
  0.00414079  0.00414079  0.00414079  0.00414079  0.00414079  0.00414079
  0.00414079  0.00414079  0.00414079  0.00414079  0.00414079  0.00414079
  0.00414079  0.00414079  0.00414079  0.00414079]


In [52]:
# log P(class) + log P(f1,1 | class) + log P(f1,2 | class) + ... + log P(f28,28 | class)

test_rough = inputDigit(name = "digitdata/testimages")

answer = np.zeros(1000)
for i in range(1000):
    test_image = np.zeros(shape=(28,28))
    for row in range(28):
        for col in range(28):
            if test_rough[i*28+row][col] == ' ':
                test_image[row][col] = 0
            else:
                test_image[row][col] = 1
             
    posteriori = np.zeros(10)
    for class_num in range(10):
        posteriori[class_num] = math.log(class_[class_num][1])
        for row in range(28):
            for col in range(28):
                if test_image[row][col] == 0:
                    posteriori[class_num] += math.log(training0[class_num][row][col])
                else:
                    posteriori[class_num] += math.log(training1[class_num][row][col] )   
    answer[i] = np.argmax(posteriori)
    print(posteriori)
    
print(answer)



[-374.62712777 -198.88076212 -261.35431941 -209.67046432 -217.35012109
 -206.38625608 -279.14709875 -149.69276872 -210.84773359 -157.57439609]
[-180.91561901 -466.88339075 -250.43472579 -241.51003033 -263.63148395
 -217.14138759 -208.49986062 -307.91120777 -238.26241168 -262.77159455]
[-314.98071036 -217.14307293 -170.21642457 -200.651217   -302.83042999
 -235.02031066 -276.28758811 -360.68054785 -173.72715593 -322.83306401]
[-278.25270522 -470.38897624 -283.61154871 -187.07419572 -334.860824
 -209.12900341 -374.7919032  -391.7741913  -206.73104613 -321.22188329]
[-352.47875676 -137.8404864  -240.02395973 -212.67622271 -218.47693949
 -187.94511022 -225.48967984 -212.96061901 -208.09988271 -206.68824216]
[-342.5264631  -237.82778884 -241.27203354 -215.58586622 -162.46926464
 -194.68550242 -239.93564606 -170.65393154 -182.03895238 -118.30296581]
[-341.4414294  -334.37914794 -314.98432297 -257.46400807 -214.67731988
 -225.46769519 -305.43479691 -129.57179975 -259.55810309 -158.1257669 ]
[

In [49]:
testlabels = readRough("digitdata/testlabels")


In [50]:
def confusion_matrix():
    num_each_class = np.zeros(10)
    conf_matrix = np.zeros(shape=(10,10))
    for i in range(1000):
        conf_matrix[int(testlabels[i])][int(answer[i])] += 1
        num_each_class[int(testlabels[i])] += 1
            
    for i in range(10):
        for j in range(10):
            conf_matrix[i][j] /= num_each_class[i]
    
    return conf_matrix

In [51]:
conf_matrix = confusion_matrix()
conf_matrix = np.around(conf_matrix, 2)
for row in conf_matrix:
    print(row)
diag_sum = 0
for i in range(10):
    diag_sum += conf_matrix[i][i]
print(diag_sum)

[ 0.84  0.    0.01  0.    0.01  0.06  0.03  0.    0.04  0.  ]
[ 0.    0.96  0.01  0.    0.    0.02  0.01  0.    0.    0.  ]
[ 0.02  0.03  0.76  0.05  0.01  0.    0.06  0.01  0.05  0.02]
[ 0.    0.02  0.    0.79  0.    0.03  0.02  0.06  0.02  0.06]
[ 0.    0.01  0.    0.    0.76  0.    0.03  0.01  0.02  0.18]
[ 0.02  0.02  0.01  0.13  0.03  0.67  0.01  0.01  0.02  0.07]
[ 0.01  0.07  0.04  0.    0.05  0.05  0.75  0.    0.02  0.  ]
[ 0.    0.07  0.03  0.    0.03  0.    0.    0.72  0.02  0.14]
[ 0.02  0.01  0.03  0.14  0.02  0.06  0.    0.01  0.6   0.12]
[ 0.01  0.01  0.01  0.03  0.09  0.02  0.    0.02  0.01  0.8 ]
7.65
